# This notebook generates data plotted in Figs. 4, 5 & 6

In [ ]:
import ipyparallel as ipp
clients = ipp.Client()

In [ ]:
clients.ids

In [ ]:
dview = clients.direct_view()

In [ ]:
with dview.sync_imports():
    import numpy as np
    import math
    #from math import exp,sqrt,log
    import numpy.random
    from scipy import signal
%px np = numpy
%px rand = numpy.random

In [ ]:
freq_range = 50 #20
freq_incr = 0.2 #0.25
mag_range = 46 #20
mag_incr = 0.1 #0.25
freq_cond = 0 # 0 or 1 -> see definition of Vin(t)
dview.push(dict(freq_range = freq_range, mag_range = mag_range, freq_incr = freq_incr, mag_incr = mag_incr, 
                freq_cond = freq_cond))

In [ ]:
def Sim_DecMak(trials,randSeed):
    
    rand.seed(randSeed)
    
    def Vin(t):   # square wave
        Vin = np.zeros(2)
        if freq_cond == 0:
            y1 = mag1*signal.square(2 * np.pi * freq1 * t, duty = 0.5)
        else:
            y1 = mag1*signal.square(2 * np.pi * freq1 * t, duty = 0.5*freq1/freq2)
        y2 = mag2*signal.square(2 * np.pi * freq2 * t, duty = 0.5)
        if y1 < 0.2:
            y1 = 0.2
        if y2 < 0.2:
            y2 = 0.2
        Vin[0] = y1
        Vin[1] = y2
        return Vin

    def sampled_stimulus(val1, val2):
        sampledStim = np.zeros(2)
        v1 = val1 + rand.normal(0, standard_dev_phys_stim)
        v2 = val2 + rand.normal(0, standard_dev_phys_stim)
        if v1 < 0.1:
            v1 = 0.1
        if v2 < 0.1:
            v2 = 0.1
        sampledStim[0] = pow(v1, gamma)
        sampledStim[1] = pow(v2, gamma)
        return sampledStim

    def Fdet_mDDM(y, Vin):    
        Fdet = np.zeros(1)
        In1 = Vin[0]
        In2 = Vin[1]
        Fdet[0] = q*(In1-In2) + drift_var_trial
        return Fdet  # 1D array

    def Frand_mDDM(stdDev):
        Frand = np.zeros(1)
        xRand0 = rand.normal(0,1)
        Frand[0] = stdDev*xRand0
        return Frand  # 1D array

    def Fdet_Cross(y, Vin):
        #linear cross-inhibition model time dependent:
        Fdet = np.zeros(2)
        Fdet[0] = -k*y[0]  - beta*y[1] + q*Vin[0]  
        Fdet[1] = -k*y[1]  - beta*y[0] + q*Vin[1]  
        return Fdet

    def Frand_Cross(stdDev):
        Frand = np.zeros(2)
        xRand0 = rand.normal(0,1)
        xRand1 = rand.normal(0,1)
        Frand[0] = stdDev*xRand0
        Frand[1] = stdDev*xRand1
        return Frand
    
    def EulerRand(dt, y, Fdet, Frand, Vin, stdDev):
        fd = Fdet(y, Vin)
        fr = Frand(stdDev)
        yNew = y + dt*fd + fr*math.sqrt(dt)
        return yNew
    
    standard_dev_phys_stim = 0.05
    gamma = 0.5
    # model parameters for linear cross inhibition
    k = 0.5 # leak, originally = 0.8
    q = 1 # conversion factor,  originally = 1
    beta = 0.25 # inhibition strength, originally =2

    delps = 0  #  phase shift between inputs
    #freq1 = 1  # frequencies of signal 1 and 2
    #freq2 = 5
    #mag1 = 2   # magnitudes of the signals
    #mag2 = 2
    #signal = np.cos(omega*time) + np.sin(2*omega*time)
    noiseCoef = 0.1 # multiplicative noise strength
    sigma = 0.1   # additive noise strength
    
    tmax = 15
    yinit = np.array([0.0, 0.0])  #initial conditions,  y is 2D array, y relates to LCA
    xinit = np.array([0.0])       # x relates to mDDM data
    ht = 0.001  #time step
    threshold = 0.6
    
    ################################################################################################
    DT_arr_av = np.zeros(freq_range*mag_range)
    ER_arr_av = np.zeros(freq_range*mag_range)
    
    freq1_list = []
    mag1_list = []
    DT_cross_list = []
    DTstd_cross_list = []
    Opt1_cross_list = []
    Opt2_cross_list = []
    Undec_cross_list = []
    DT_mddm_list = []
    DTstd_mddm_list = []
    Opt1_mddm_list = []
    Opt2_mddm_list = []
    Undec_mddm_list = []
    for ii in range(freq_range):
        freq = 0.5 + freq_incr*ii
        freq1 = freq
        freq2 = 2/3*freq
        for kk in range(mag_range):
            mag = 0.5 + mag_incr*kk
            mag1 = mag
            mag2= 3/4*mag
            Opt1_cross = 0
            Opt2_cross = 0
            Undec_cross = 0
            Opt1_mddm = 0
            Opt2_mddm = 0
            Undec_mddm = 0
            DT_cross = []
            DT_mddm = []
            for nn in range(trials):
                t = 0
                y = yinit + rand.uniform(0,0.2,2)
                Input = Vin(t)
                InTransform = sampled_stimulus(Input[0], Input[1])
                while t < tmax and y[0] < 2.0*threshold and y[1] < 2.0*threshold:  
                    stdDev = sigma+0.2
                    y = EulerRand(ht, y, Fdet_Cross, Frand_Cross, InTransform, stdDev) 
                    for j1 in range(len(y)):
                        y[j1] = max(0, y[j1])
                    t = t + ht
                    Input = Vin(t)
                    InTransform = sampled_stimulus(Input[0], Input[1])
                if t < tmax:
                    DT_cross.append(t)
                    if y[0] > y[1]:
                        Opt1_cross += 1
                    elif y[1] > y[0]:
                        Opt2_cross += 1
                else:
                    Undec_cross += 1
                
                drift_var_trial = rand.normal(0,0.1)
                t = 0
                x = xinit + rand.uniform(-0.1,0.1)
                Input = Vin(t)
                InTransform = sampled_stimulus(Input[0], Input[1])
                while t < tmax and abs(x) < threshold: 
                    variance_tot = (sigma*sigma) + noiseCoef*(InTransform[0]**2 + InTransform[1]**2)
                    stdDev = math.sqrt(variance_tot) 
                    x = EulerRand(ht, x, Fdet_mDDM, Frand_mDDM, InTransform, stdDev)
                    t = t + ht
                    Input = Vin(t)
                    InTransform = sampled_stimulus(Input[0], Input[1])
                if t < tmax:
                    DT_mddm.append(t)
                    if x > 0:
                        Opt1_mddm += 1
                    elif x < 0:
                        Opt2_mddm += 1
                else:
                    Undec_mddm += 1
                    
            freq1_list.append(freq1)
            mag1_list.append(mag1)
            DT_cross_list.append(np.mean(np.asarray(DT_cross)))
            DTstd_cross_list.append(np.std(np.asarray(DT_cross)))
            Opt1_cross_list.append(Opt1_cross/trials)
            Opt2_cross_list.append(Opt2_cross/trials)
            Undec_cross_list.append(Undec_cross/trials)
            DT_mddm_list.append(np.mean(np.asarray(DT_mddm)))
            DTstd_mddm_list.append(np.std(np.asarray(DT_mddm)))
            Opt1_mddm_list.append(Opt1_mddm/trials)
            Opt2_mddm_list.append(Opt2_mddm/trials)
            Undec_mddm_list.append(Undec_mddm/trials)
            
        
    return freq1_list, mag1_list, np.asarray(DT_cross_list), np.asarray(DTstd_cross_list),\
np.asarray(Opt1_cross_list), np.asarray(Opt2_cross_list), np.asarray(Undec_cross_list), np.asarray(DT_mddm_list),\
np.asarray(DTstd_mddm_list), np.asarray(Opt1_mddm_list), np.asarray(Opt2_mddm_list), np.asarray(Undec_mddm_list)

In [ ]:
import timeit
time_start = timeit.default_timer()
nr_engines = len(clients.ids)
randSeedList = [(jj+1)*1928374 for jj in range(nr_engines)]
nr_trials_tot = 2000 #1000
nr_trials_per_engine = int(nr_trials_tot/nr_engines)
results = dview.map_sync(Sim_DecMak, [nr_trials_per_engine]*nr_engines, randSeedList)
time_stop = timeit.default_timer()
print('elapsed time =',time_stop - time_start,'seconds')

In [ ]:
freq1_list = results[0][0]
mag1_list = results[0][1]
#print(freq_ratio_list)
#print(mag_ratio_list)
DT_cross_array_av = np.zeros(freq_range*mag_range)
DTstd_cross_array_av = np.zeros(freq_range*mag_range)
Opt1_cross_array_av = np.zeros(freq_range*mag_range)
Opt2_cross_array_av = np.zeros(freq_range*mag_range)
Undec_cross_array_av = np.zeros(freq_range*mag_range)
DT_mddm_array_av = np.zeros(freq_range*mag_range)
DTstd_mddm_array_av = np.zeros(freq_range*mag_range)
Opt1_mddm_array_av = np.zeros(freq_range*mag_range)
Opt2_mddm_array_av = np.zeros(freq_range*mag_range)
Undec_mddm_array_av = np.zeros(freq_range*mag_range)
for kk in range(nr_engines):
    DT_cross_array_av += results[kk][2]/nr_engines
    DTstd_cross_array_av += results[kk][3]/nr_engines
    Opt1_cross_array_av += results[kk][4]/nr_engines
    Opt2_cross_array_av += results[kk][5]/nr_engines
    Undec_cross_array_av += results[kk][6]/nr_engines
    DT_mddm_array_av += results[kk][7]/nr_engines
    DTstd_mddm_array_av += results[kk][8]/nr_engines
    Opt1_mddm_array_av += results[kk][9]/nr_engines
    Opt2_mddm_array_av += results[kk][10]/nr_engines
    Undec_mddm_array_av += results[kk][11]/nr_engines

with open("SimDecTimeErrorRate_FreqRatio_3over2_MagRatio_4over3_FreqCondNull.csv", "w") as out:
    out.write("#f1, m1, meanDTcross, stdDTcross, Opt1_cross, Opt2_cross, Undec_cross, meanDTmddm, stdDTmddm, Opt1_mddm, Opt2_mdd, Undec_mddm\n")
    for ii in range(len(freq1_list)):
        out.write(str(freq1_list[ii])+','+str(mag1_list[ii])+','+str(DT_cross_array_av[ii])+','
                  +str(DTstd_cross_array_av[ii])+','+str(Opt1_cross_array_av[ii])+','+str(Opt2_cross_array_av[ii])+','
                  +str(Undec_cross_array_av[ii])+','+str(DT_mddm_array_av[ii])+','+str(DTstd_mddm_array_av[ii])+','
                  +str(Opt1_mddm_array_av[ii])+','+str(Opt2_mddm_array_av[ii])+','+str(Undec_mddm_array_av[ii])+'\n')